## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# Import API key
from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barrow,US,71.2906,-156.7887,-20.18,69,100,10.36,light snow
1,1,Bredasdorp,ZA,-34.5322,20.0403,62.28,83,100,6.46,overcast clouds
2,2,Jalingo,NG,8.8833,11.3667,72.00,16,78,5.32,broken clouds
3,3,Albany,US,42.6001,-73.9662,40.68,68,99,2.93,overcast clouds
4,4,Lashio,MM,22.9333,97.7500,64.18,41,3,1.16,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
go_to_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
go_to_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Jalingo,NG,8.8833,11.3667,72.00,16,78,5.32,broken clouds
7,7,Touros,BR,-5.1989,-35.4608,79.70,72,85,8.34,overcast clouds
8,8,Mamallapuram,IN,12.6264,80.1722,78.85,65,41,8.16,scattered clouds
11,11,Kirakira,SB,-10.4544,161.9205,82.99,80,98,11.05,overcast clouds
13,13,Bubaque,GW,11.2833,-15.8333,77.22,71,4,4.25,clear sky
17,17,Kahului,US,20.8947,-156.4700,80.62,70,40,11.50,scattered clouds
22,22,Hambantota,LK,6.1241,81.1185,79.34,69,73,3.36,broken clouds
23,23,Luderitz,NaN,-26.6481,15.1594,73.38,51,81,17.31,broken clouds
27,27,Rikitea,PF,-23.1203,-134.9692,79.05,73,37,17.20,scattered clouds
30,30,Beloha,MG,-25.1667,45.0500,70.65,91,6,4.07,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
go_to_cities_df.count()

City_ID                189
City                   189
Country                186
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Current Description    189
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = go_to_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Jalingo,NG,72.00,broken clouds,8.8833,11.3667,
7,Touros,BR,79.70,overcast clouds,-5.1989,-35.4608,
8,Mamallapuram,IN,78.85,scattered clouds,12.6264,80.1722,
11,Kirakira,SB,82.99,overcast clouds,-10.4544,161.9205,
13,Bubaque,GW,77.22,clear sky,11.2833,-15.8333,
17,Kahului,US,80.62,scattered clouds,20.8947,-156.4700,
22,Hambantota,LK,79.34,broken clouds,6.1241,81.1185,
23,Luderitz,NaN,73.38,broken clouds,-26.6481,15.1594,
27,Rikitea,PF,79.05,scattered clouds,-23.1203,-134.9692,
30,Beloha,MG,70.65,clear sky,-25.1667,45.0500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace = True)
hotel_df.dropna(subset = ["Hotel Name"], inplace = True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Jalingo,NG,72.00,broken clouds,8.8833,11.3667,Debright Motel
7,Touros,BR,79.70,overcast clouds,-5.1989,-35.4608,INN NEW HORIZON
8,Mamallapuram,IN,78.85,scattered clouds,12.6264,80.1722,Radisson Blu Resort Temple Bay Mamallapuram
11,Kirakira,SB,82.99,overcast clouds,-10.4544,161.9205,SanBiz Lodge
13,Bubaque,GW,77.22,clear sky,11.2833,-15.8333,"Casa Dora, Bubaque"
...,...,...,...,...,...,...,...
533,Adelaide,AU,79.41,few clouds,-34.9333,138.6000,The Chancellor on Currie
537,Garissa,KE,72.99,broken clouds,-0.4536,39.6401,Sharif Plaza Hotel
540,Pamanukan,ID,78.30,overcast clouds,-6.2842,107.8106,Paris Day Hotel Pamanukan
541,Inirida,CO,72.59,overcast clouds,3.8653,-67.9239,Fuente del Guania Hotel de Lujo


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))